# Edge-Vertex Continuous Collision Detection (CCD)

Given a point that moves over time, $\vec{p}_0(t)$, and an edge, $\vec{e}(s, t)$, comprised of two points, $\vec{p}_1(t)$ and $\vec{p}_2(t)$, we want to find the time of impact between the point and edge. Let us assume a normalized time step $t=0$ to $t=1$. We known the point positions for $t=0$ and $t=1$. Let us also assume that for all points the trajectory is linear and the velocity is constant for all points. We can write the trajectories as 

\begin{align}
    \vec{p}_i(t) &= [\vec{p}_i(1) - \vec{p}_i(0)] t + \vec{p}_i(0) & \forall i \in \{0, 1, 2\}, t \in [0, 1]\\
    e(s, t) &= (\vec{p}_2(t) - \vec{p}_1(t)) s + \vec{p}_1(t) & \forall s \in [0, 1] 
\end{align}

In [1]:
# Compute a polynomial for the intersection of a point and edge both moving
# through time.
# General idea: http://www.sci.utah.edu/~kpotter/publications/ramsey-2004-RBPI.pdf

import numpy
import sympy
from IPython.display import display, Markdown, Math, Latex
from sympy.printing.latex import latex
import string
# sympy.init_printing(use_latex='mathjax')

# s is the spatial parameterization for the edge
# t is the temporal parameterization
s, t = sympy.symbols("s t")

def display_eq(name, value):
    display(Math(f"{name}:={latex(value)}"))

In [2]:
dimensions = 2  # Number of dimensions of the points and edge
# For each dimenstion write the trajectory of the point and edge
d = "y" if dimensions == 2 else "z"
# Three points, one free and two for the edge, each with a velocity
p = numpy.reshape(sympy.symbols(f"p_0:3_x:{d}", real=True), (-1, dimensions))
v = numpy.reshape(sympy.symbols(f"v_0:3_x:{d}", real=True), (-1, dimensions))

In [3]:
# Write the point's trajectory as a line in time
pt = p + t * v
# Write the dimensions of the edge
e = (pt[2,:] - pt[1,:]) * s + pt[1,:]

In [4]:
sn = [None] * dimensions
sd = [None] * dimensions
seq = [None] * dimensions
for i in range(0, dimensions):
    # Equate the point and edge in the current dimension
    eq = sympy.Eq(pt[0,i], e[i])
    # Solve for the spatial parameterization along the edge
    s_ = sympy.solve(eq, s)[0]
    display_eq(f"s_{i}", s_)
    # Split the spatial parameterization into a numerator and denominator
    # NB: The denominator can be zero if the edge is degenerate
    sn[i], sd[i] = sympy.fraction(s_)

# Equate the different dimensions of the spatial parameterization
if dimensions == 2:
    eq1 = sympy.Eq(sn[0] * sd[1], sn[1] * sd[0])
elif dimensions == 3:
    eq1 = sympy.Eq(sd[2] * (sn[0] * sd[1] - sn[1] * sd[0]), sn[0])
else:
    raise Exception("Invalid dimension option")

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [5]:
# Write the equations as a polynomial in t
t_poly = sympy.polys.polytools.poly_from_expr(eq1, t)

# Display the coefficents
for coeff_name, coeff_val in zip(string.ascii_lowercase, t_poly[0].all_coeffs()):
    display_eq(coeff_name, sympy.factor(coeff_val))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Equation for $t$ in 2D:
We want to solve the quadratic equation $a t^2 + b t + c = 0$ with the coefficient defined above.

Using the formula $$t = \frac{-b \pm \sqrt{b^2 - 4 a c}}{2a}$$ is unstable for values of $a$ near zero.

Another alternative is the **Citardauq Formula** $$t = \frac{-2c}{ |b| \pm \sqrt{b^2 - 4 a c}}$$
where one of the roots is stable when $a \rightarrow 0$. The other root goes to infinitity (as $-b + |b|$) goes to zero.

What we will do is use one or the other one depending on the values of $a$ and $b$.

In [6]:
from math import sqrt
def solve_quadratic_v1(a, b, c):
    D = b**2 - 4*a*c
    x1, x2 = None, None
    if D > 0:
        if a != 0:
            x1 = (-b + sqrt(D)) / (2 * a)
            x2 = (-b - sqrt(D)) / (2 * a)
        elif b != 0:
            x1 = - c / b
    return x1, x2

def solve_quadratic_v2(a, b, c):
    D = b**2 - 4*a*c
    x1, x2 = None, None
    if D > 0:
        if b > 0:
            x1 = -2 * c / (b + sqrt(D))
            if a != 0 :
                x2 = (-b - sqrt(D))/(2*a)
        else:
            x2 = -2*c/(-b + sqrt(D))
            if a != 0:
                x1 = (-b + sqrt(D))/(2*a)
#     elif a == b == c == 0:
#         x1, x2 = numpy.inf, numpy.inf
    return x1, x2

In [7]:
x = solve_quadratic_v1(2.0, 1.0, -1.0)
y = solve_quadratic_v2(2.0, 1.0, -1.0)
print(x, y)

(0.5, -1.0) (0.5, -1.0)


In [8]:
print(solve_quadratic_v2(0, 0, 0))

(None, None)


## Minimum Seperation Distance

We want to impose a minimum speration distance between the vertex and edge. We solve for the time at which the point and edge are at a distance $h$ away from each other. The distance between the point and edges, $d(t)$, is

\begin{align}
    s(t) &= \min\left(\max\left(\frac{\left(\vec{p}_0(t) - \vec{p}_1(t)\right) \cdot \left(\vec{p}_2(t) - \vec{p}_1(t)\right)}{\| \vec{p}_2(t) - \vec{p}_1(t) \|^2}, 0\right), 1\right)\\
    \vec{p}_*(t) &= \vec{p}_1(t) + s(t) * (\vec{p}_2(t) - \vec{p}_1(t))\\
    d(t) &= \|\vec{p}_0(t) - \vec{p}_*(t)\|
\end{align}

where $\vec{p}_*(t)$ is the closest point on the edge to $\vec{p}_0(t)$ at time $t$. To find the time at which the point is a distance $h$ away from the edge, we solve the equation $d(t) - h = 0$ or equivalently $d(t)^2 - h^2 = 0$. 

In [9]:
# h is the minimum seperation distance
h = sympy.symbols("h")

# Compute the spatial parameter of the closest point
st = ((pt[0] - pt[1]).dot(pt[2] - pt[1])) / ((pt[2] - pt[1]).dot(pt[2] - pt[1]))

# Compute the closest point along the edge
pt_closest = pt[1] + st * (pt[2] - pt[1])

# Compute the distance between p0 and p*
dt_sq = (pt[0] - pt_closest).dot(pt[0] - pt_closest)
dt_sq = sympy.simplify(dt_sq)
display_eq("d(t)^2", dt_sq)

<IPython.core.display.Math object>

In [10]:
msd_eq = sympy.Eq(sympy.fraction(dt_sq)[0], h**2 * sympy.fraction(dt_sq)[1])

# Write the equations as a polynomial in t
msd_t_poly = sympy.polys.polytools.poly_from_expr(msd_eq, t)

In [12]:
# Display the coefficients
# for coeff_name, coeff_val in zip(string.ascii_lowercase, msd_t_poly[0].all_coeffs()):
#     display_eq(coeff_name, sympy.factor(coeff_val))
print("The equation for the time of minimum spereration is a degree {} polynomial.".format(
    len(msd_t_poly[0].all_coeffs()) - 1))

The equation for the time of minimum spereration is a degree 6 polynomial.
